In [11]:
import importlib
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.utils import gen_batches
import numpy as np
from Code.SGDPLS import SGDPLS
from Code.OLPLS import OLPLS
from Code.CIPLS import CIPLS
from Code.SIMPLS import SIMPLS
from Code.IPLS import IPLS
from Code.PLS1 import IPLS1
# ----------
from sklearn.cross_decomposition import PLSRegression
from pytictoc import TicToc
tictoc=TicToc()

In [3]:
dataset='CTSlices'
CT_dat = np.genfromtxt("data/CTSlices/slice_localization_data.csv",
                       delimiter=",",
                       skip_header=1)
X_train = CT_dat[:,1:-1]
y_train = CT_dat[:,-1]

n_train = X_train.shape[0]
batch_size = 100
batch_num = int(n_train/batch_size)
print(n_train, batch_num, batch_size)
trainset = [ slice((i+1)*batch_size) for i in range(batch_num)]

In [28]:
ypred_all = {}
tlst = {'samples': [(i+1)*batch_size for i in range(batch_num)]}

In [21]:
def RunExp(pls,timer):
    for ind,batch in enumerate(gen_batches(n_train,batch_size)):
        tictoc.tic()
        pls.fit(X_train[batch], y_train[batch])
        timer[ind] = tictoc.tocvalue()
    ypred = pls.predict(X_train[:])

    return timer,ypred

In [29]:
pls = PLSRegression(n_components=7, scale=False).\
    fit(X_train[:n_train,:],y_train[:n_train])
pls_pred = pls.predict(X_train[:n_train]).ravel()
ypred_all.update({'PLRegression': pls_pred})

In [33]:
n_comp=7
timer = np.zeros((batch_num, ))
pls = OLPLS(n_components=n_comp,amnesic=0.99,mu=1e-4)
timer,ypred = RunExp(pls,timer)
ypred_all.update({'OLPLS': ypred.ravel()})
tlst.update({'OLPLS': timer})

In [ ]:
n_comp=8
timer = np.zeros((batch_num, ))
pls = SIMPLS(n_components=n_comp)
timer,ypred = RunExp(pls,timer)
ypred_all.update({'SIMPLS': ypred.ravel()})
tlst.update({'SIMPLS': timer})

In [ ]:
n_comp=7
timer = np.zeros((batch_num, ))
pls = IPLS1(n_components=n_comp)
timer,ypred = RunExp(pls,timer)
ypred_all.update({'IPLS1': ypred.ravel()})
tlst.update({'IPLS1': timer})

In [34]:
n_comp=7
timer = np.zeros((batch_num, ))
pls = SGDPLS(n_components=n_comp,epochs=100,eta=1e-4)
timer,ypred = RunExp(pls,timer)
ypred_all.update({'SGDPLS': ypred.ravel()})
tlst.update({'SGDPL': timer})

In [35]:
print(ypred_all)

{'PLRegression': array([18.06983939, 19.83481596, 20.57304785, ..., 41.05752271,
       12.81999267, 11.3007853 ]), 'OLPLS': array([40.41111774, 40.63270067, 40.68833153, ..., 47.38210415,
       44.99744747, 45.28198677])}
